In [ ]:
from langchain_openai import ChatOpenAI
import os
api_key=os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(api_key=api_key)


In [9]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing in several ways, including:\n\n1. Automated testing: Langsmith can be used to create automated test scripts that can be run repeatedly to check for bugs and issues in the software.\n\n2. Performance testing: Langsmith can be used to simulate high volumes of traffic on a website or application to test its performance under different conditions.\n\n3. Integration testing: Langsmith can be used to test how different components of a system work together to ensure they are functioning correctly.\n\n4. Load testing: Langsmith can be used to test how a system performs under heavy loads to identify any bottlenecks or performance issues.\n\n5. Regression testing: Langsmith can be used to re-run tests on a system after changes have been made to ensure that no new bugs have been introduced.\n\nOverall, Langsmith can help streamline the testing process and improve the quality of software by identifying and fixing issues early on in the development

In [10]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])
chain = prompt | llm 

In [11]:
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content='Langsmith can help with testing in several ways:\n\n1. **Automated Testing**: Langsmith can be used to generate automated test cases for software applications. By providing input parameters and expected output for different scenarios, Langsmith can help in creating a comprehensive set of test cases that can be automated to run repeatedly.\n\n2. **Load Testing**: Langsmith can generate synthetic load on a system by creating a large number of virtual users making requests simultaneously. This can help in testing the performance and scalability of the system under different load conditions.\n\n3. **Security Testing**: Langsmith can be used to generate various types of security attacks such as SQL injection, cross-site scripting, and others to test the security vulnerabilities of an application.\n\n4. **Regression Testing**: Langsmith can help in automating regression testing by generating test cases for new features or changes in the system and running them alongside ex

In [12]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content='Langsmith can help with testing in several ways:\n\n1. Test Automation: Langsmith can be used to automate testing processes, making it easier to run tests repeatedly and consistently. This can help reduce manual effort and improve the efficiency of the testing process.\n\n2. Test Data Generation: Langsmith can be used to generate test data for different scenarios, helping testers cover a wide range of test cases and improve test coverage.\n\n3. Performance Testing: Langsmith can be used to simulate large numbers of users or data to test the performance of a system under various conditions. This can help identify potential bottlenecks and optimize performance.\n\n4. Integration Testing: Langsmith can be used to simulate interactions between different components or systems to ensure that they work together seamlessly. This can help identify integration issues early in the development process.\n\n5. Security Testing: Langsmith can be used to simulate security attacks or

In [13]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [14]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [15]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [16]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [17]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [18]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

LangSmith can help with testing by allowing developers to create datasets, run tests on their LLM applications, upload test cases in bulk, create test cases on the fly, export test cases from application traces, run custom evaluations, compare different versions of applications, provide a playground environment for rapid iteration and experimentation, enable beta testing to collect real-world performance data, capture feedback from users, annotate traces for evaluation criteria, add runs as examples to datasets, provide monitoring charts for tracking key metrics over time, perform online evaluations and automations on production traces in near real-time, and group traces from a single conversation for easier tracking and annotation.


In [19]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [20]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.Prototyping‚ÄãPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing ‚Äî and debug where it is failing ‚Äî is incredibly important for this phase.Debugging‚ÄãWhen developing new LLM applications, we suggest having LangSmith tracing enabl

In [21]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [22]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.Prototyping‚ÄãPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing ‚Äî and debug